# Asssignment 3

In [3]:
import spotipy
import json

from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth
from kafka import KafkaProducer, KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [1]:
packages = {
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0"
}

In [4]:
load_dotenv()
spark = SparkSession.builder.appName('assignment3').config("spark.jars.packages", ",".join(packages)).getOrCreate()
producer = KafkaProducer(bootstrap_servers='localhost:9092')

23/05/25 18:59:24 WARN Utils: Your hostname, HP-Elite-845 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface wlan0)
23/05/25 18:59:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hajta2/.local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hajta2/.ivy2/cache
The jars for the packages stored in: /home/hajta2/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-496098ff-ed48-40f5-bbd3-1b152e8405c9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 710ms :: artifacts dl 25ms
	:: 

In [5]:
scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [7]:
producer.send('tracks_topic', b'')
producer.send('audio_features_topic', b'')
df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "tracks_topic, audio_features_topic")
    .option("startingOffsets", "latest")
    .option("maxRatePerPartition", 50)
    .load()
)

In [8]:
tracks_stream = df.selectExpr("CAST(value AS STRING)").filter("topic = 'tracks_topic'")
features_stream = df.selectExpr("CAST(value AS STRING)").filter("topic = 'audio_features_topic'")

In [9]:
tracks_query = tracks_stream.writeStream.format("memory").queryName("tracks").start()
features_query = features_stream.writeStream.format("memory").queryName("features").start()

23/05/25 19:02:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f3dbd400-5061-41be-93c2-9c23304846db. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/25 19:02:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/05/25 19:02:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-efccd5ed-4a46-4a99-b5eb-13959c3a3dc0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/25 19:02:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

In [10]:
saved_tracks = sp.current_user_saved_tracks(limit=50)
for item in saved_tracks['items']:
    track = item['track']
    producer.send('tracks_topic', json.dumps(track).encode('utf-8'))
    audio_features = sp.audio_features(track['id'])
    producer.send('audio_features_topic', json.dumps(audio_features[0]).encode('utf-8'))

In [19]:
tracks = spark.sql("select * from tracks")
features = spark.sql("select * from features")

In [13]:
tracks_query.stop()
features_query.stop()

In [35]:
tracks.printSchema()
features.printSchema()

root
 |-- value: string (nullable = true)

root
 |-- value: string (nullable = true)



In [32]:
tracks_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("artists", StringType(), True),
        StructField("duration_ms", StringType(), True),
        StructField("popularity", StringType(), True),
    ]
)

tracks_parsed = tracks.withColumn(
    "parsed_value", from_json(col("value"), tracks_schema)
).select("parsed_value.*")

In [33]:
tracks_parsed.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- popularity: string (nullable = true)



In [38]:
features_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("acousticness", StringType(), True),
        StructField("energy", StringType(), True),
        StructField("key", StringType(), True),
        StructField("loudness", StringType(), True),
        StructField("mode", StringType(), True),
        StructField("speechiness", StringType(), True),
        StructField("acousticness", StringType(), True),
        StructField("instrumentalness", StringType(), True),
        StructField("liveness", StringType(), True),
        StructField("valence", StringType(), True),
        StructField("tempo", StringType(), True),
    ]
)

features_parsed = features.withColumn(
    "parsed_value", from_json(col("value"), features_schema)
).select("parsed_value.*")

In [39]:
features_parsed.printSchema()

root
 |-- id: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)



In [40]:
joined = tracks_parsed.join(features_parsed, tracks_parsed.id == features_parsed.id).drop(features_parsed.id)

In [41]:
joined.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)

